# dataset/ingest

In [9]:
import os
from tqdm import *
from abcli import file
from abcli.modules import objects
from abcli import fullname
from abcli.plugins import cache
from roofAI.semseg.utils import visualize
from roofAI.dataset import RoofAIDataset, DatasetKind, MatrixKind
from roofAI.dataset.ingest.AIRS import ingest_AIRS
from IPython.display import Image
from roofAI import NAME, VERSION
from abcli import string
from abcli.plugins import seed
import numpy as np

import abcli.logging
import logging

logger = logging.getLogger()

logger.info(f"{fullname()}, built on {string.pretty_date()}")

abcli-7.2383.1-2023-06-aws-batch-a, built on 21 October 2023, 12:55:32


In [2]:
cache_object_name = cache.read("roofAI_ingest_AIRS_cache")
ingest_object_name=objects.unique_object("roofAI-AIRS-ingest")

📂 roofAI-AIRS-ingest-2023-10-21-12-54-48-78614


In [3]:
cache_object_path = objects.object_path(cache_object_name)
ingest_object_path = objects.object_path(ingest_object_name)
logger.info(f"-> {ingest_object_path}")

-> /Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-54-48-78614


In [4]:
counts = {
    "test": 10,
    "train": 10,
    "val": 10
}

chip_width = 480
chip_height = 360
chip_overlap = 0.1

log=True

In [5]:
cache_dataset = RoofAIDataset(cache_object_path)
ingest_dataset = RoofAIDataset(ingest_object_path, kind=DatasetKind.CAMVID).create(log=log)

RoofAIDataset[DatasetKind.AIRS:AIRS](roofAI_ingest_AIRS_cache_2023-10-15-15-53-41-09195): 3 subset(s): 95 X test + 857 X train + 94 X val
RoofAIDataset[DatasetKind.CAMVID:AIRS](roofAI-AIRS-ingest-2023-10-21-12-54-48-78614): 3 subset(s): 0 X test + 0 X train + 0 X val
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-54-48-78614/SegNet-Tutorial/CamVid/test)
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-54-48-78614/SegNet-Tutorial/CamVid/testannot)
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-54-48-78614/SegNet-Tutorial/CamVid/train)
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-54-48-78614/SegNet-Tutorial/CamVid/trainannot)
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-54-48-78614/SegNet-Tutorial/CamVid/val)
abcli.path.create(/Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-21-12-54-48-78614/SegNet-Tutorial/Ca

In [6]:
def slice_matrix(
    input_matrix: np.ndarray,
    kind: MatrixKind,
    chip_height: int,
    chip_width: int,
    chip_overlap: float,
    max_chip_count: int,
    path: str,
    prefix:str,
    log:bool=False,
) -> int:
    if log:
        logger.info("slice_matrix[{}]: {} -{}X{}x{}-@{:.0f}%-> {} - {}".format(
            string.pretty_shape_of_matrix(input_matrix),
            kind,
            max_chip_count,
            chip_height,
            chip_width,
            chip_overlap * 100,
            path,
            prefix,
        ))

    count = 0
    for y in range(0,input_matrix.shape[0] - chip_height, int(chip_overlap * chip_height)):
        for x in range(0,input_matrix.shape[1] - chip_width, int(chip_overlap * chip_width)):
            count += 1

            chip = input_matrix[
                y:y+chip_height,
                x:x+chip_width
                ]

            assert file.save_image(
                os.path.join(
                    path,
                    "{}-{:010d}-{:010d}.png".format(
                        prefix,
                        y,
                        x,
                    )
                ),
                chip,
                log=log,
            )

            if count >= max_chip_count:
                return count

    return count

In [7]:
log_ = False
for subset in tqdm(counts.keys()):
    for matrix_kind in list(MatrixKind):
        chip_count = counts[subset]
        for record_id in cache_dataset.subsets[subset]:
            input_matrix = cache_dataset.get_matrix(
                subset,
                record_id,
                matrix_kind,
                log=log_,
            )

            chip_count -= slice_matrix(
                input_matrix,
                matrix_kind,
                chip_height,
                chip_width,
                chip_overlap,
                max_chip_count=chip_count,
                path=ingest_dataset.subset_path(subset,matrix_kind),
                prefix=record_id,
                log=log_,
            )

            if chip_count <= 0:
                break

100%|██████████| 3/3 [00:10<00:00,  3.39s/it]


In [8]:
file.save_yaml(
        os.path.join(ingest_dataset.path, "metadata.yaml"),
        {
            "classes": Dataset.CLASSES,
            "path_prefix": "SegNet-Tutorial/CamVid",
            "source": "CamVid",
            "ingested-by": f"{NAME}-{VERSION}",
        },
        log=True,
    )


NameError: name 'ingest_path' is not defined

In [ ]:
ingest_dataset = RoofAIDataset(ingest_object_path)

```python
assert ingest_AIRS(
    cache_path=cache_object_path,
    ingest_path=ingest_object_path,
    {
        "test": 10,
        "train": 10,
        "val": 10
    }
)
```

In [ ]:
seed.log(ingest_object_path)

In [ ]:
# END